# Day 11

## Part I

变体的康威生命游戏。按照标准写法来做，此处没有使用Numpy这样的库，因此没有应用矢量化函数进行运算，最终运行性能比较差。因此这里的代码主要为了清晰易懂而不是性能。

下面首先定义一个枚举类型，代表每个格子的可能状态，有三种空座位empty、被占座位occupied和地板floor：

In [1]:
from enum import Enum

CellState = Enum('CellState', ('empty', 'occupied', 'floor'))

然后是为了完成生命游戏的核心逻辑代码，此处如果应用Numpy数组和相应的矢量化函数运算的话，应该能提升不少性能，但是就失去了使用enum的代码清晰易懂特性：

In [2]:
from typing import List

def occupy_seats_count(cells: List[List[CellState]], row: int, col: int) -> int:
    '''
    计算一个单元格周围被占用的座位个数
    '''
    drdc = ((dr, dc) for dr in range(-1, 2) for dc in range(-1, 2) if not (dr == 0 and dc == 0))
    count = 0
    for dr, dc in drdc:
        r = row + dr
        c = col + dc
        # 超出范围的座位忽略
        if r < 0 or c < 0 or r >= len(cells) or c >= len(cells[row]):
            continue
        if cells[r][c] == CellState.occupied:
            count += 1
    return count

def next_round(cells: List[List[CellState]]) -> List[List[CellState]]:
    '''
    计算下一轮的整个地图情况，这里没有采用双缓冲，而是直接返回了一个新的List
    '''
    result = []
    for row, line in enumerate(cells):
        new_line = []
        for col, cell in enumerate(line):
            neighbor_occupied = occupy_seats_count(cells, row, col)
            # 如果当前未被占用且周围占用情况为0，则占该座位
            if cell == CellState.empty and neighbor_occupied == 0:
                new_line.append(CellState.occupied)
            # 如果被占用且周围占用数量大于等于4，则取消占用
            elif cell == CellState.occupied and neighbor_occupied >= 4:
                new_line.append(CellState.empty)
            else:
                new_line.append(cell)
        result.append(new_line)
    return result

读取输入，返回一个CellState的二维列表：

In [3]:
def read_input(input_file: str) -> List[List[CellState]]:
    cells = []
    with open(input_file) as fn:
        line = fn.readline()
        while line:
            row = []
            for c in line.rstrip():
                if c == 'L':
                    row.append(CellState.empty)
                if c == '#':
                    row.append(CellState.occupied)
                if c == '.':
                    row.append(CellState.floor)
            cells.append(row)
            line = fn.readline()
    return cells

定义一个函数用来计算整个地图被占用座位的总数：

In [4]:
def count_all_occupied(cells: List[List[CellState]]) -> int:
    return sum(len([s for s in row if s == CellState.occupied]) for row in cells)

第一部分问题的实现逻辑，一直运行直到前后两次的地图完全相同，这里可以采取更加有效的方式，通过一个标记记录地图是否发生了修改来判断结束条件，因为这需要定义自己的数据结构，而且Python本来性能就不会很好，因此未采纳，可在Rust版本中看到相应实现：

In [5]:
def part1_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells)

单元测试：

In [6]:
testcase = read_input('testcase1.txt')
assert(part1_solution(testcase) == 37)

获得第一部分结果：

In [7]:
cells = read_input('input.txt')
part1_solution(cells)

2310

## Part II

第二部分需要重新定义计算周围座位被占用个数的方法。首先一共有八个不同的方向，每个方向都从距离当前座位最近的开始，直到遇到一个被占用或者空闲为止，下面的one_drection函数即完成这个工作。然后direction_occupied_count用来计算全部八个方向的总占用数量：

In [8]:
def one_direction(cells: List[List[CellState]], row: int, col: int, init_dr: int, init_dc: int) -> int:
    dr, dc = init_dr, init_dc
    while 0 <= row + dr < len(cells) and 0 <= col + dc < len(cells[row]):
        r, c = row + dr, col + dc
        if cells[r][c] == CellState.occupied:
            return 1
        if cells[r][c] == CellState.empty:
            return 0
        dr, dc = dr + init_dr, dc + init_dc
    return 0

def direction_occupied_count(cells: List[List[CellState]], row: int, col: int) -> int:
    count = 0
    drdc = ((dr, dc) for dr in range(-1, 2) for dc in range(-1, 2) if not (dr == 0 and dc == 0))
    for dr, dc in drdc:
        count += one_direction(cells, row, col, dr, dc)
    return count

重新定义下一轮next_round函数，增加counter参数作为用来计算周围座位占用的函数，增加thresh参数，作为判断周围占用座位数量的极限值：

In [9]:
def next_round(cells: List[List[CellState]], counter: callable, thresh: int) -> List[CellState]:
    result = []
    for row, line in enumerate(cells):
        new_line = []
        for col, cell in enumerate(line):
            neighbor_occupied = counter(cells, row, col)
            if cell == CellState.empty and neighbor_occupied == 0:
                new_line.append(CellState.occupied)
            elif cell == CellState.occupied and neighbor_occupied >= thresh:
                new_line.append(CellState.empty)
            else:
                new_line.append(cell)
        result.append(new_line)
    return result

然后是第二部分实现逻辑，使用direction_occupied_count函数和阈值5代入next_round，获得结果：

In [11]:
def part2_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells, direction_occupied_count, 5)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells, direction_occupied_count, 5)

单元测试：

In [12]:
part2_solution(testcase)

26

第二部分结果：

In [13]:
part2_solution(cells)

2074

可以重新定义第一部分实现逻辑，将occupy_seats_count函数和阈值4代入next_round，实现两个部分逻辑统一：

In [14]:
def part1_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells, occupy_seats_count, 4)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells, occupy_seats_count, 4)

重新计算第一部分结果：

In [15]:
part1_solution(cells)

2310